In [ ]:
# coding: utf-8

'''Script to invert Tongariro SP data using streaming current density

from Soegi Kang, Dominique Fournier, UBC, Craig Miller, GNS

https://github.com/simpeg

please use the SP_spherical branch

'''

from SimPEG import Utils, Maps, Mesh, Directives
from SimPEG.EM.Static import SP
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
from SimPEG.EM.Static.Utils import StaticUtils
import os
from sklearn.metrics import mean_squared_error as mse
from matplotlib.ticker import AutoMinorLocator
import colorcet as cc
from SimPEG.Utils import download
import tarfile

tar_folder = download( "https://storage.googleapis.com/simpeg/miller_et_al_2018.tar.gz")  
tar = tarfile.open(tar_folder, "r:*")
tar.extractall()
tar.close()

datadir ="./miller_et_al_2018/"

plt.close('all')

In [ ]:
#Load data, conductivity and topo files

topo = np.loadtxt(datadir + 'topoxyz.dat', skiprows=1)
data = np.loadtxt(datadir + 'Tongariro_SP_data.dat')

cond_model_file = datadir + 'MT_model_for_SP_inv.con'
mesh_file = datadir + 'Tongariro_SP.msh'

dataInv=data.copy()


In [ ]:
#read in and set up mesh

topo = topo[(topo[:,0] < 1833000) & (topo[:,0] > 1824000)]
topo = topo[(topo[:,1] > 5662000) & (topo[:,1] < 5670000)]

topoInv = topo

#set reference electrode location
xref, yref = 1830000, 5666000

#Read in the mesh
mesh = Mesh.TensorMesh.readUBC(mesh_file)

print( mesh)
print( 'width in X direction', mesh.hx.sum(), 'm')
print( 'width in Y direction', mesh.hy.sum(), 'm')
print( 'width in Z direction', mesh.hz.sum(), 'm')
print( 'number of cells', mesh.nC)


In [ ]:
#sigma is background conductivity (S/m)

#read in conducitivity model file
resistivity = mesh.readModelUBC(cond_model_file) #read background conductivity model from file
sigma = 1/resistivity #change to S/m from ohmm
actind = ~(sigma==1e-8)
# sets uniform conductivity background
#sigma = np.ones(mesh.nC)*1e-3
# sigma[~actind] = 1e-8 #sigma of cells above topo = 1e-8


In [ ]:
#set active cells
actMap = Maps.InjectActiveCells(mesh, actind, 0.) #makes cell below topo active and sets cells above topo to 0
actMap_amp = Maps.InjectActiveCells(mesh, actind, -100.) #makes cell below topo active and sets cells above topo to 0
actMap_plot = Maps.InjectActiveCells(mesh, actind, np.nan)
mesh2D, topoCC = StaticUtils.gettopoCC(mesh, actind) #~actind = opposite of actind, ie swaps True for False



In [ ]:
#drape points onto topo
xyzlocInv = StaticUtils.drapeTopotoLoc(mesh, dataInv[:,:2], actind=actind)
xyzlocInv = np.c_[xyzlocInv[:,:2], xyzlocInv[:,2]]


In [ ]:
# Depth weighting    # weight=0. to remove depth weighting.

weight = 2.
zlocCC = Utils.mkvc(topoCC.reshape([-1,1]).repeat(mesh.nCz, axis=1))
depthweight = 1./ ((abs(mesh.gridCC[:,2] - zlocCC)+50.)**weight)
depthweight /= depthweight.max()
depthweight[~actind] = np.nan
zlocCC[~actind] = np.nan

#np.savetxt(datadir + 'Locs.xyz', xyzlocInv)


In [ ]:
#setup the survey

from pymatsolver import PardisoSolver

ind = np.argmin(abs(xyzlocInv[:, 0]-xref)+abs(xyzlocInv[:, 1]-yref))
inds = np.ones(xyzlocInv.shape[0], dtype='bool')
inds[ind] = False
xyzM = xyzlocInv[inds, :]
xyzN = np.atleast_2d(xyzlocInv[ind]).repeat(xyzM.shape[0], axis=0)


wires = Maps.Wires(('jsx', actMap.nP), ('jsy', actMap.nP), ('jsz', actMap.nP))
prob = SP.Problem_CC_Jstore(mesh, sigma=sigma, jsxMap=actMap*wires.jsx, jsyMap=actMap*wires.jsy, jszMap=actMap*wires.jsz,
                            Solver=PardisoSolver)
rx = SP.Rx.Dipole(xyzM, xyzN)
src = SP.Src.StreamingCurrents([rx], mesh=mesh, modelType="CurrentDensity", indActive=actind)
survey = SP.SurveySP_store([src])
survey.pair(prob)

survey.dobs = (dataInv[1:,2] - dataInv[0,2])


In [ ]:
#for cartesian 

from SimPEG import (Mesh, Maps, Utils, DataMisfit, Regularization,
                    Optimization, Inversion, InvProblem)

#set noise floor
survey.std = 30 #survey std dev in  mV
#survey.eps = abs(survey.dobs).max() * 0.05
survey.eps = 60 # survey noise floor in mV


dmisfit = DataMisfit.l2_DataMisfit(survey)
regmap = Maps.IdentityMap(nP = actMap.nP*3)


reg_jsx = Regularization.Sparse(mesh, mapping=wires.jsx, indActive=actind)
reg_jsy = Regularization.Sparse(mesh, mapping=wires.jsy, indActive=actind)
reg_jsz = Regularization.Sparse(mesh, mapping=wires.jsz, indActive=actind)


reg = reg_jsx + reg_jsy + reg_jsz
opt = Optimization.ProjectedGNCG(maxIter=100, maxIterCG=10, tolX=1e-20, tolF=1e-20)
invProb = InvProblem.BaseInvProblem(dmisfit, reg, opt)
target = Directives.TargetMisfit()


In [ ]:
# Create an inversion object
# run the cartesian L2 inversion.

update_Jacobi = Directives.UpdatePreconditioner()
update_SensWeight = Directives.UpdateSensitivityWeights()

beta = Directives.BetaSchedule(coolingFactor=8, coolingRate=3)
betaest = Directives.BetaEstimate_ByEig()

#L2 only
inv = Inversion.BaseInversion(invProb, directiveList=[beta, betaest, update_SensWeight, target, update_Jacobi])


prob.counter = opt.counter = Utils.Counter()
opt.LSshorten = 0.5

m0 = np.ones(actMap.nP*3)*0.  #creates an initial starting model m0
reg.mref = m0
mopt_L2 = inv.run(m0) #mopt is the L2 output model


In [ ]:
#############################################
# LAST STEP: Finish inversion with spherical formulation for sparsity
# Lp inversion starts here
# It uses the cartesian L2 model from above as a starting model
############################################

#beta = invProb.beta
beta = 1.

mstart = Utils.matutils.xyz2atp(mopt_L2)
mref = np.zeros_like(mstart)

prob.coordinate_system = 'spherical'
prob.model = mstart

# Create a block diagonal regularization
wires = Maps.Wires(('amp', actMap.nP), ('theta', actMap.nP), ('phi', actMap.nP))

# Create a regularization
reg_a = Regularization.Sparse(mesh, indActive=actind, mapping=wires.amp)
reg_a.norms = [0, 2, 2, 2]
#reg_a.eps_p = np.percentile(np.abs(mstart[:actMap.nP]), 95)
reg_a.mref = mref

reg_t = Regularization.Sparse(mesh, indActive=actind, mapping=wires.theta)
reg_t.alpha_s = 0.  # No reference angle
reg_t.coordinate_system = 'spherical'
reg_t.norms = [2, 2, 2, 2] #0,1,1,1
#reg_t.eps_q = 1e-2 # in radians
reg_t.mref = mref
# reg_t.alpha_x, reg_t.alpha_y, reg_t.alpha_z = 0.25, 0.25, 0.25

reg_p = Regularization.Sparse(mesh, indActive=actind, mapping=wires.phi)
reg_p.alpha_s = 0.  # No reference angle
reg_p.coordinate_system = 'spherical'
reg_p.norms = [2, 2, 2, 2] #0,1,1,1
#reg_p.eps_q = 1e-2 # in radians
reg_p.mref = mref

reg = reg_a + reg_t + reg_p
reg.mref = mref


In [ ]:
#Run Lp inversion
# Data misfit function
dmisfit = DataMisfit.l2_DataMisfit(survey)

Lbound = np.kron(np.asarray([0., -np.inf, -np.inf]), np.ones(actMap.nP))
Ubound = np.kron(np.asarray([np.inf, np.inf, np.inf]), np.ones(actMap.nP))

# Add directives to the inversion
opt = Optimization.ProjectedGNCG(maxIter=30,
                                 lower=Lbound,
                                 upper=Ubound,
                                 maxIterLS=10,
                                 maxIterCG=20, tolCG=1e-3,
                                 stepOffBoundsFact=1e-8)
#store each iteration
opt.remember('xc')

# Here is where the norms are applied
IRLS = Directives.Update_IRLS(f_min_change=1e-4,
                              minGNiter=2, beta_tol=1e-2,
                              coolingRate=3, maxIRLSiter=10)

invProb = InvProblem.BaseInvProblem(dmisfit, reg, opt, beta=beta)

# Special directive specific to the spherical problem. The sensitivity
# weights are update between each iteration.
update_SensWeight = Directives.UpdateSensitivityWeights()

ProjSpherical = Directives.ProjSpherical()

update_Jacobi = Directives.UpdatePreconditioner()
betaest = Directives.BetaEstimate_ByEig(beta0_ratio=0.1)
inv = Inversion.BaseInversion(invProb, directiveList=[ProjSpherical, IRLS, update_SensWeight, update_Jacobi])

# #start the inversion
# moptSpherical_Lp = inv.run(mstart)

# #recall a specific iteration of the inversion. Start counting at zero!.
# #Comment out if not needed
# xc = opt.recall('xc')
# moptSpherical_Lp = xc[3]


In [ ]:
# #Write out Lp model Spherical formulation (cartesian Lp doesnt exist)
# mout = Utils.matutils.atp2xyz(moptSpherical_Lp)
# temp = mout.reshape((actMap.nP, 3), order="F")

# SP.Utils.writeVectorUBC(mesh, datadir + 'Tongairo_S_Lp.fld', actMap.P * temp)

# mesh.writeModelUBC(datadir + 'Tongairo_S_Lp_amp.mod', actMap_amp * (wires.amp * moptSpherical_Lp))

# #Write calculated data (Lp model)
# Lp_calculated = survey.dpred(moptSpherical_Lp)
# np.savetxt('L2_calculated_data.dat', Lp_calculated, fmt='%.4f')

# RMS = np.sqrt(mse(Lp_calculated, survey.dobs))
# print('RMS = %.3f mV' %RMS)


In [ ]:
# # Plot  Lp model obs and calc
# plt.figure()
# plt.plot(survey.dobs, '.', label = 'obs')
# plt.plot(invProb.dpred, label = 'calc')
# plt.legend()
# plt.ylabel('mV')
# plt.xlabel('data point index')
# plt.annotate('A', xy=(0, 1000),fontsize=14)
# plt.title('$J_s$ observed vs calculated.  RMS = %.0f mV' %RMS)
# plt.savefig('D:\Dropbox\phd\Writing\SP_paper\working\Supporting_information\Figures\Js_Obs_vs_calc.png')
# plt.savefig('D:\Dropbox\phd\Writing\SP_paper\working\Supporting_information\Figures\Js_Obs_vs_calc.eps')

# plt.figure()
# plt.hist(moptSpherical_Lp, bins=100, label='Lp model')
# #plt.xlim(-0.002, 0.002)
# plt.legend()
# plt.show()

In [ ]:
# # Convert spherical to cartesian
# moptSpherical_Lp_cart = Utils.matutils.atp2xyz(moptSpherical_Lp)
# j_spherical_lp = actMap.P * moptSpherical_Lp_cart.reshape((actMap.nP, 3), order="F")
# j_spherical_lp[~actind, :] = np.nan
# j_amp = np.ones(mesh.nC)*np.nan
# j_amp[actind] = wires.amp*moptSpherical_Lp

# # extract core mesh
# xmin, xmax = dataInv[:,0].min(), dataInv[:,0].max()
# ymin, ymax = dataInv[:,1].min(), dataInv[:,1].max()
# zmin, zmax = 700, topo[:,2].max()
# xyzlim = np.array([[xmin, xmax], [ymin, ymax], [zmin, zmax]])
# ind_core, mesh_core = Utils.ExtractCoreMesh(xyzlim, mesh)

# j_spherical_lp_core = Utils.mkvc(j_spherical_lp[ind_core, :])
# actind_core = actind[ind_core]
# jamp_min, jamp_max = j_spherical_lp_core[:mesh_core.nC][actind_core].min(), j_spherical_lp_core[:mesh_core.nC][actind_core].max()
# jamp_lp_core = j_amp[ind_core]
# jamp_lp_core[~actind_core] = np.nan


In [ ]:
# # plot map views
# fig, ((ax0,ax1),(ax2,ax3)) = plt.subplots(nrows=2, ncols=2, figsize = (10, 10))

# fig.subplots_adjust(wspace=0, hspace=0)
# axs = [ax0, ax1, ax2, ax3]
# layers = [1500, 1200, 1000, 800]


# clim = (0, 0.08)
# for i_layer, ax in enumerate(axs):   
#     indz = int(np.argmin(abs(mesh_core.vectorCCz-layers[i_layer])))
#     cbar = mesh_core.plotSlice(
#            j_spherical_lp_core, ind = indz, clim=clim, normal="Z", ax=ax,
#            pcolorOpts={'cmap':'cet_diverging_rainbow_bgymr_45_85_c67'},
#            streamOpts={'color':'k', 'linewidth':0.5, 'density':0.8},
#            vType='CCv', view='vec')
    
#     minorLocator = AutoMinorLocator(4)
#     ax.xaxis.set_minor_locator(minorLocator)
#     ax.yaxis.set_minor_locator(minorLocator)

#     ax.tick_params(which='both', width=2, direction='in')
#     ax.tick_params(which='major', length=10, direction='in', top= True, right=True)
#     ax.tick_params(which='minor', length=7, direction='in', top = True)
    
#     ax.set_title('')
#     ax.set_xlabel('')
#     ax.set_ylabel('')
#     ax.set_aspect('equal')
#     ax.set_ylim(5663000, 5669000)
#     ax.set_xlim(1825094, 1831794)    
#     ax.set_title(("%.0f")%(layers[i_layer]))
#     ax.set_xticks(np.arange(1826000, 1832000, 2000))
#     ax.set_yticks(np.arange(5663000, 5670000, 2000))
    
#     if ~np.logical_or(i_layer==2,i_layer==3):
#         ax.set_xticklabels([])
#     if np.logical_or(i_layer==1,i_layer==3):
#         ax.set_yticklabels({})

# plt.tight_layout()
# plt.show()


In [ ]:
# #plot section views
# fig, ((ax0,ax1),(ax2,ax3),(ax4,ax5)) = plt.subplots(
#     nrows=3, ncols=2, figsize = (23, 10) #23,10
# )

# fig.delaxes(ax5)

# fig.subplots_adjust(wspace=0, hspace=0)
# axs = [ax0, ax1, ax2, ax3, ax4, ax5]
# layers = [5667994, 5666994, 5665994, 5664994, 5663994]


# for i_layer, ax in enumerate(axs[:-1]):
#     indy = int(np.argmin(abs(mesh_core.vectorCCy-layers[i_layer])))
    
#     cbar = mesh_core.plotSlice(
#            j_spherical_lp_core, ind = indy, clim=clim, normal="Y", ax=ax,
#            pcolorOpts={'cmap':'cet_diverging_rainbow_bgymr_45_85_c67'},
#            streamOpts={'color':'k', 'linewidth':1, 'density':0.8},
#            vType='CCv', view='vec')

#     minorLocator = AutoMinorLocator(4)
#     ax.xaxis.set_minor_locator(minorLocator)
#     #ax.yaxis.set_minor_locator(minorLocator)

#     ax.tick_params(which='both', width=2, direction='in')
#     ax.tick_params(which='major', length=10, direction='in', top= True, right=True)
#     ax.tick_params(which='minor', length=7, direction='in', top = True)
    
#     ax.set_title('')
#     ax.set_xlabel('')
#     ax.set_ylabel('')
#     #ax.set_aspect('equal')
#     ax.set_ylim(700, 2000)
#     ax.set_xlim(1825094, 1831794)    
#     #ax.set_title(("%.0f")%(layers[i_layer]))
#     ax.set_xticks([1826000, 1828000, 1830000])
#     ax.set_yticks([1000, 1500, 2000])
    
    
    
#     if ~np.logical_or(i_layer==4,i_layer==3) :
#         ax.set_xticklabels([])
#     if np.logical_or(i_layer==1,i_layer==3):
#         ax.set_yticklabels({})
        
# plt.tight_layout()
# plt.show()

In [ ]:
# # Write core of the LP model to file in spherical formulation
# mesh_core.writeUBC('mesh_core.msh')
# model_vec = j_spherical_lp_core.reshape((mesh_core.nC, 3), order='F')
# model_vec[~actind_core, :]=-100
# SP.Utils.writeVectorUBC(mesh_core, 'Tongairo_S_LP_core.fld', model_vec)

# jamp_lp_core = j_amp[ind_core]
# jamp_lp_core[~actind_core] = -100
# mesh_core.writeModelUBC('Tongairo_S_LP_amp_core.mod', jamp_lp_core)
